In [ ]:
import tensorflow as tf
import pandas as pd
tf.__version__

In [2]:
feature_df = pd.read_csv("../input_data/CSVs/letter-recognition.data", header=None)
feature_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
1,0,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
2,1,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
3,2,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
4,3,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8


In [3]:
target_df = feature_df.pop(0)
target_df.head()

0    0
1    0
2    1
3    2
4    3
Name: 0, dtype: int64

In [4]:
# Split DataFrame into training and testing - first 15000 are training, remaining 500 are testing
train_features = feature_df.iloc[:15000]
train_target = target_df.iloc[:15000]
test_features = feature_df.iloc[15000:]
test_target = target_df.iloc[15000:]
# Test that operation worked successfully by ensuring output row count is right, and that the test set starts at 15000
test_features.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
15000,6,11,6,8,3,4,13,9,2,10,6,3,1,10,4,8
15001,4,9,5,7,3,6,7,7,8,9,8,10,2,10,4,9
15002,12,15,10,8,5,5,3,4,5,4,7,6,6,6,2,7
15003,4,8,6,6,5,4,7,1,6,10,9,11,3,8,3,6
15004,5,8,6,6,5,7,8,4,7,7,6,8,7,8,3,7


In [5]:
# Ensure output is reproducible
tf.random.set_seed(8)

# Create a new model
model = tf.keras.models.Sequential()

# Create layers to use for the model - all fully connected and using relu to keep between 0-1, before using softmax for categorical output
fc1 = tf.keras.layers.Dense(512, input_shape=(16,), activation='relu')
fc2 = tf.keras.layers.Dense(512, activation='relu')
fc3 = tf.keras.layers.Dense(128, activation='relu')
fc4 = tf.keras.layers.Dense(128, activation='relu')
fc5 = tf.keras.layers.Dense(26, activation='softmax')

# Add the layers to the model
model.add(fc1)
model.add(fc2)
model.add(fc3)
model.add(fc4)
model.add(fc5)

# Display a summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               8704      
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 128)               65664     
                                                                 
 dense_3 (Dense)             (None, 128)               16512     
                                                                 
 dense_4 (Dense)             (None, 26)                3354      
                                                                 
Total params: 356890 (1.36 MB)
Trainable params: 356890 (1.36 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
# Set up the model to be trained to output only for the character it believes is correct
loss = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(0.001)

# Compile the model using the selected functions
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
# Train the model over 5 epochs
model.fit(train_features, train_target, epochs=5)

Epoch 1/5
469/469 [==============================] - 5s 7ms/step - loss: 1.2013 - accuracy: 0.6367
Epoch 2/5
469/469 [==============================] - 3s 7ms/step - loss: 0.5849 - accuracy: 0.8142
Epoch 3/5
469/469 [==============================] - 3s 6ms/step - loss: 0.4116 - accuracy: 0.8703
Epoch 4/5
469/469 [==============================] - 3s 6ms/step - loss: 0.3324 - accuracy: 0.8890
Epoch 5/5
469/469 [==============================] - 3s 6ms/step - loss: 0.2821 - accuracy: 0.9040


In [7]:
# Test the model on the testing dataset
model.evaluate(test_features, test_target)

157/157 [==============================] - 1s 2ms/step - loss: 0.3457 - accuracy: 0.8904


[0.34567543864250183, 0.8904219269752502]

In [9]:
# Predict the classes (characters) of the test dataset
preds_proba = model.predict(test_features)
# COnvert the set if predictions to a single value per prediction
preds = preds_proba.argmax(axis=1)
# Generate a confusion matrix for the predictions
print(tf.math.confusion_matrix(test_target, preds))

157/157 [==============================] - 0s 2ms/step
tf.Tensor(
[[182   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0
    0   0   0   0   0   0   0   1]
 [  0 178   2   0   0   0   1   0   8   0   1   0   0  11   2   0   0   0
    0   2   0   0   0   0   0   0]
 [  1   0 208   0   0   0   3   0   0   0   0   0   0   3   0   1   0   0
    0   0   0   0   0   0   0   0]
 [  0   0   9 171   0   0   0   0   0   2   0   6   0   0   0   2   3   0
    0   0   1   0   0   3   1   0]
 [  1   0   2   0 169   0   4   0   0   2   0   2   5   1   3   1   1   2
    0   1   1   0   7   1   5   0]
 [  1   0   0   0   2 159   2   0   0   0   3   0   0   1   0   0   0   1
    0   5   0   0   7   0   0  17]
 [  4   1   1   0   0   1 166   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0]
 [  0   0   0   0   1   0   1 185   0   1   1   0   5   1   0   1   0   0
    0   0   0   4   3   2   1   0]
 [  0   9   4   0   0   2   0   0 165   0   0   1   0   0   0 

# We can see that the model is performing well, as the output table has the highest values mostly following the diagonal - the diagonal of this table represents correct predictions, so high values along this line with now values elsewhere shows high accuracy with few mistakes.